# <font color='blue'>ALPAR - Governo Digital - Gerador de Dados</font>

In [ ]:
# install conector MySQL
# !pip install mysql-connector-python

#install pyDF2
# !pip install PyPDF2

#install timestring
#!pip install timestring


In [ ]:
import pandas as pd
import numpy as np
import copy
import csv
import codecs
import random as rd
import string
import timestring as ts
import datetime as dt

print("pandas versão", pd.__version__)

pd.options.display.max_rows = 2000

# <font color='black'>parâmetros gerais</font>

In [ ]:
# parâmetros cadastrais
pr = {
    'entidade': {
        'min' : 30, 
        'max': 50
    },
    'servico': {
        'tot': 200,                # lista total de serviços (de 1 a n)
        'min': 60,                 # número mínimo de serviços que fará parte da simulação
        'max': 100                 # número máximo de serviços que fará parte da simulação
    },
    'categoria_servico': {
        'tot': 100,                # lista total de categoria de serviços (de 1 a n)
        'min': 4,                  # número mínimo de categoria de serviços que fará parte da simulação
        'max': 7                   # número máximo de categoria de serviços que fará parte da simulação
    },
    'grupo_responsavel': {
        'tot': 100,                # lista total de grupos responsáveis (de 1 a n)
        'min': 2,                  # número mínimo de grupo responsável que fará parte da simulação
        'max': 6                   # número máximo de grupo responsável que fará parte da simulação
    },
    'usuario': {
        'tot': 100,                # lista total de usuários (de 1 a n)
        'min': 30,                 # número mínimo de usuários que fará parte da simulação
        'max': 40                  # número máximo de usuários que fará parte da simulação
    },
    'grupo_usuario': {
        'tot': 50,                 # lista total de grupos de usuários (de 1 a n)
        'min': 10,                 # número mínimo de grupos de usuários que fará parte da simulação
        'max': 14                  # número máximo de grupos de usuários que fará parte da simulação
    },
    'departamento': {
        'tot': 100,                # lista total de departamentos de encaminhamento (de 1 a n)
        'min': 10,                 # número mínimo de departamentos de encaminhamento que fará parte da simulação
        'max': 14                  # número máximo de departamentos de encaminhamento que fará parte da simulação
    },    
    'servicos_por_entidade': {
        'min': 10,                 # número mínimo de serviços por entidade
        'max': 20                  # número máximo de serviços por entidade
    },
    'acao': {
        'tot': 100,                # lista total de ações (de 1 a n)
        'min': 10,                 # número mínimo de ações por entidade
        'max': 20                  # número máximo de ações por entidade
    },
    'categorias_servico_por_entidade': {
        'min': 5,                  # número mínimo de categorias de serviço por entidade
        'max': 7                   # número máximo de categorias de serviço por entidade
    },
    'usuários_por_entidade': {
        'min': 10,                 # número mínimo de usuários por entidade
        'max': 20                  # número máximo de usuários por entidade
    },
    'grupos_usuários_por_entidade': {
        'min': 10,                 # número mínimo de grupos de usuários por entidade
        'max': 20                  # número máximo de grupos de usuários por entidade
    },
    'tasks_por_protocolo': {
        'cancelado': {
            'min': 2,                              # número mínimo de grupos de usuários por entidade
            'max': 8,                              # número máximo de grupos de usuários por entidade
            'nome': 'Cancelado',                   # nome da situação
            'nome_task': 'Protocolo Cancelado'     # nome da última task da coleção
        },
        'concluido': {
            'min': 5,                              # número mínimo de grupos de usuários por entidade
            'max': 12,                             # número máximo de grupos de usuários por entidade
            'nome': 'Concluído',                   # nome da situação
            'nome_task': 'Protocolo Concluído'     # nome da última task da coleção
        },
        'andamento': {
            'min': 2,                              # número mínimo de grupos de usuários por entidade
            'max': 11,                             # número máximo de grupos de usuários por entidade
            'nome': 'Em Andamento',                # nome da situação
            'nome_task': '<Sem Ação>'              # nome da última task da coleção
        },
        'tempos': {                                # tempos mínimos e máximos a serem estipulados (em dias)
            'min': 0.5,
            'max': 10
        },
        'gap_entre_tasks': {
            'min': 0.05,
            'max': 1.0
        },
        'gap_atendimento': {
            'min': 0.05,
            'max': 0.1,
            'prop_anda_atend': (3, 7)       # (x, y) proporção média de com recebimento (a cada x, y é recebido)
        }
    }
}

In [ ]:
# parâmetros transacionais
pr_trans = {
    'protocolo': {
        'min' : 300,                  # quantidade mínima de protocolos por entidade entidade
        'max': 3000,                  # quantidade máxima de protocolos por entidade entidade
        'prop_status': (20, 5, 5),     # proporção de status de protocolo - (em andamento, cancelado, concluído)
        'prop_desc_status': ('Em Andamento', 'Cancelado', 'Concluido'),
#         'prop_cancelados': (20, 1),   # (x, y) proporção média de protocolos cancelados (a cada x, y é cancelado)
#         'prop_encerrados': (10, 1),   # (x, y) proporção média de protocolos encerrados (a cada x, y é encerrado)
        'data_inicial': (2020, 1, 1), # data inicial da massa de dados
        'dias_ant_ult_task': {        # qtd de dias antes de hoje
            'min': 3,
            'max': 6
        },
        'ult_dia': 3                  # qtd de dias antes de hoje 
    }
}

# <font color='black'>funções usadas</font>

In [ ]:
def Ret_Situacao(a_concluido, a_cancelado):
    if a_concluido == 1:
        sit = (pr['tasks_por_protocolo']['concluido']['nome'],
               pr['tasks_por_protocolo']['concluido']['nome_task'],
               pr['tasks_por_protocolo']['concluido']['min'],
               pr['tasks_por_protocolo']['concluido']['max']
              )
    elif a_cancelado == 1:
        sit = (pr['tasks_por_protocolo']['cancelado']['nome'],
               pr['tasks_por_protocolo']['cancelado']['nome_task'],
               pr['tasks_por_protocolo']['cancelado']['min'],
               pr['tasks_por_protocolo']['cancelado']['max']
              )
    else:
        sit = (pr['tasks_por_protocolo']['andamento']['nome'],
               pr['tasks_por_protocolo']['andamento']['nome_task'],
               pr['tasks_por_protocolo']['andamento']['min'],
               pr['tasks_por_protocolo']['andamento']['max']
              )
    return sit

# <font color='black'>lê arquivos externos</font>

In [ ]:
# abrindo os arquivos...
caminho = "./dados_externos/"
arq_cidades = caminho + "brazil_cities_coordinates.csv"

df_cidades = pd.read_csv(arq_cidades, sep = ',', dtype = str)
print("linhas = %s , colunas = %s " % df_cidades.shape)

lst_cidades = list(df_cidades['city_name'])

# <font color='black'>gera Entidades</font>

In [ ]:
# estipula a quantidade de entidades fará parte do dataset
qtd_entidades = rd.randint(pr['entidade']['min'],pr['entidade']['max'])

# gera a lista de cidades
lst_tmp_entidades = rd.sample(lst_cidades, k = qtd_entidades)
# lst_tmp_entidades

In [ ]:
# define orgãos de governo
lst_tmp_orgao = ['Prefeitura de ', 'Secretaria de ']

# gera a lista de entidades
lst_entidades = [rd.choices(lst_tmp_orgao, [4, 1], k =1)[0] + sub for sub in lst_tmp_entidades]
# lst_entidades

# <font color='black'>gera Serviços</font>

## <font color='green'>gera Grupos Responsável</font>

In [ ]:
# gera a lista total de grupos responsáveis que será usada
lst_tmp_grupo_responsavel = []
for i in range(pr['grupo_responsavel']['tot']):
    lst_tmp_grupo_responsavel.append('Grupo Responsável ' + str(i + 1))
# lst_tmp_grupo_responsavel

## <font color='green'>gera Categoria de Serviços</font>

In [ ]:
# gera a lista total de grupos de serviços que será usada
lst_tmp_categoria_servico = []
for i in range(pr['categoria_servico']['tot']):
    lst_tmp_categoria_servico.append('Categoria de Servico ' + str(i + 1))
# lst_tmp_categoria_servico

## <font color='green'>gera Serviços</font>

In [ ]:
# gera a lista total de serviços que será usada
lst_tmp_servico = []
for i in range(pr['servico']['tot']):
    lst_tmp_servico.append('Servico ' + str(i + 1))
# lst_tmp_servico

# <font color='black'>gera Usuários</font>

## <font color='green'>gera Grupos de Usuários</font>

In [ ]:
# gera a lista total de grupos de usuários que será usada
lst_tmp_grupo_usuario = []
for i in range(pr['grupo_usuario']['tot']):
    lst_tmp_grupo_usuario.append('Grupo de Usuário ' + str(i + 1))
# lst_tmp_grupo_usuario

## <font color='green'>gera Usuários</font>

In [ ]:
# gera a lista total de usuários que será usada
lst_tmp_usuario = []
for i in range(pr['usuario']['tot']):
    lst_tmp_usuario.append('Usuário ' + str(i + 1))
# lst_tmp_usuario

# <font color='black'>gera Departamentos (encaminhado para)</font>

## <font color='green'>gera Departamentos</font>

In [ ]:
# gera a lista total de departamentos que será usada na coluna "encaminhado para"
lst_tmp_departamento = []
for i in range(pr['departamento']['tot']):
    lst_tmp_departamento.append('Departamento ' + str(i + 1))
# lst_tmp_departamento

# <font color='black'>gera Ações</font>

## <font color='green'>gera acões</font>

In [ ]:
# gera a lista total de ações que será usada
lst_tmp_acao = []
for i in range(pr['acao']['tot']):
    lst_tmp_acao.append('Ação ' + str(i + 1))
# lst_tmp_acao

# <font color='black'>Cria estrutura Entidade --> serviços, usuários, departamentos </font>

In [ ]:
lst = []

# para cada entidade gera uma coleção de serviços e de usuários
for i in lst_entidades:
    
# serviços
# estipula a quantidade de serviços será cadastrado para esta entidade
    qtd = rd.randint(pr['servicos_por_entidade']['min'], pr['servicos_por_entidade']['max'])
# define a coleção de serviços para esta entidade
    lst_servicos_da_entidade = rd.sample(lst_tmp_servico, k = qtd)
    
# estipula a quantidade de grupos responsáveis será cadastrado para esta entidade
    qtd = rd.randint(pr['grupo_responsavel']['min'], pr['grupo_responsavel']['max'])
# define a coleção de grupos responsáveis para esta entidade
    lst_grupos_responsaveis_da_entidade = rd.sample(lst_tmp_grupo_responsavel, k = qtd)

    # estipula a quantidade de categorias de serviços será cadastrado para esta entidade
    qtd = rd.randint(pr['categoria_servico']['min'], pr['categoria_servico']['max'])
# define a coleção de categorias de serviços para esta entidade
    lst_categoria_servicos_da_entidade = rd.sample(lst_tmp_categoria_servico, k = qtd)

# escolhe o grupo de responsavel e a categoria de servicos por cada serviço
    lst_serv = []
    for j in lst_servicos_da_entidade:
        dic_serv = {
            'servico': j,
            'grupo_resp': rd.sample(lst_grupos_responsaveis_da_entidade, k = 1)[0],
            'categoria_serv': rd.sample(lst_categoria_servicos_da_entidade, k = 1)[0]
        }
        lst_serv.append(dic_serv)

# usuários
# estipula a quantidade de usuários será cadastrado para esta entidade
    qtd = rd.randint(pr['usuários_por_entidade']['min'], pr['usuários_por_entidade']['max'])
# define a coleção de serviços para esta entidade
    lst_usuarios_da_entidade = rd.sample(lst_tmp_usuario, k = qtd)

# estipula a quantidade de grupos de usuários será cadastrado para esta entidade
    qtd = rd.randint(pr['grupos_usuários_por_entidade']['min'], pr['grupos_usuários_por_entidade']['max'])
# define a coleção de grupos de usuários para esta entidade
    lst_grupos_usuario_da_entidade = rd.sample(lst_tmp_grupo_usuario, k = qtd)
    
# escolhe o grupo de usuários por cada usuário
    lst_user = []
    for j in lst_usuarios_da_entidade:
        dic_user = {
            'usuario': j,
            'grupo_usuario': rd.sample(lst_grupos_usuario_da_entidade, k = 1)[0]
        }
        lst_user.append(dic_user)  
        
# departamentos
# estipula a quantidade de departamentos será cadastrado para esta entidade
    qtd = rd.randint(pr['departamento']['min'], pr['departamento']['max'])
# define a coleção de serviços para esta entidade
    lst_departamentos = rd.sample(lst_tmp_departamento, k = qtd)

# ações
# estipula a quantidade de ações que será cadastrado para esta entidade
    qtd = rd.randint(pr['acao']['min'], pr['acao']['max'])
# define a coleção de serviços para esta entidade
    lst_acoes = rd.sample(lst_tmp_acao, k = qtd)
# define qual será a ação que terá solicitação de apoio
    acao_apoio = rd.sample(lst_acoes, k = 1)[0]
# refaz a lista de serviço para uma lista de tuplas: (serviço, 0 = não tem apoio ou 1 = tem apoio)
    lst_acoes = [(i, 1 if i == acao_apoio else 0) for i in lst_acoes]


# monta estrutura cadastral de 1 entidade para n serviços e n usuários
    dic = {'entidade': i,
           'servicos': lst_serv,
           'usuarios': lst_user,
           'acoes': lst_acoes,
           'departamentos': lst_departamentos
          }
    lst.append(dic)
# lst

# <font color='black'>gera Protocolos</font>

In [ ]:
# calcula a quantidade de dias entre a data inicial e hoje
dt_ini = dt.date(pr_trans['protocolo']['data_inicial'][0],
                  pr_trans['protocolo']['data_inicial'][1],
                  pr_trans['protocolo']['data_inicial'][2])
hoje = dt.datetime.now().date()
dias = (hoje - dt_ini).days - pr_trans['protocolo']['ult_dia']

In [ ]:
# estipula a data do protocolo
dt_protocolo = dt_ini + dt.timedelta(days = rd.randint(0, dias))

In [ ]:
# parâmetros da função:
# p = registro do protocolo
# ac = lista de ações a serem soreteadas
# t_min_max = tupla com o tempo mínimo e tempo máximo para sorteio

def Gera_Task(p, deps, ac, tempos, gaps, gap_atend):
    lst_task = []
    cont_task = 0
    tot_task = p['qtd_tasks']
    hoje = dt.datetime.now()
    
##########################################################################################################
# PRIMEIRA AÇÃO - acrescenta uma primeira ação em todos os protocolos
    dic = {'seq_task': 1,
           'acao': ('Abertura de Processo Administrativo', 0),
           'processo_encerrado': 0,
           'processo_cancelado': 0
          }
    lst_task.append(dic)
    cont_task += 1
    
##########################################################################################################
#  AÇÕES INTERMEDIÁRIAS - gera tasks intermediárias (sem a possibilidade de ações iguais consecutivas)

    acao_anterior = None
    for i in range(2, tot_task):
        
        while 1:  # loop para evitar que sorteie acões iguais consecutivas
            ac_e = rd.sample(ac, k = 1)
            if ac_e[0] != acao_anterior:
                break
        acao_anterior = ac_e[0]
        
        dic = {'seq_task': i,
               'acao': ac_e[0],
               'processo_encerrado': 0,
               'processo_cancelado': 0
              }
        lst_task.append(dic)

##########################################################################################################
# ÚLTIMA AÇÃO - se for um protocolo concluído ou cancelado coloca task de conclusão ou cancelamento

    
    dic = {'seq_task': tot_task,
           'acao': (p['situacao'][1], 0),
           'processo_encerrado': 1 if (p['encerrado'] == 1 or p['cancelado'] == 1) else 0,
           'processo_cancelado': p['cancelado']
          }
    lst_task.append(dic)
    cont_task += 1
    
##########################################################################################################
# COMPLEMENTA A COLEÇÃO DE AÇÕES
    
    # define a data da última task
    total_tasks = len(lst_task)                                             # total de tasks
    dias = rd.randint(pr_trans['protocolo']['dias_ant_ult_task']['min'],    # total de dias para tirar de hoje
                      pr_trans['protocolo']['dias_ant_ult_task']['min']
                     )
    dt_ult_task = hoje + dt.timedelta(days = -dias)                         # data de conclusão da última task

    # calcula as datas de todas as tasks
    andamento = 1 if p['situacao'][0] == 'Em Andamento' else 0              # flag de task em andamento

    # determina se a task mais recente de um protocolo em andamento tem data de recebimento
    tem_dt_recebimento = rd.choices([0, 1], [gap_atend[2][0], gap_atend[2][1]], k =1)[0]

    ult_dt_conclusão = dt_ult_task
    for k in range(total_tasks - 1, -1, -1):

        tp = rd.uniform(tempos[0], tempos[1])                           # sorteia o tempo da task
        ult_dt_inicio = ult_dt_conclusão + dt.timedelta(days = -tp)
        gap = rd.uniform(gaps[0], gaps[1])                              # sorteia o gap entra uma task e outra task        

        gap_atendimento = rd.uniform(gap_atend[0], gap_atend[1])    # sorteia o gap entre criação da task e o atendimento
        dt_atendimento = ult_dt_inicio + dt.timedelta(days = gap_atendimento)
        
        lst_task[k]['DT_INICIO'] = ult_dt_inicio
        lst_task[k]['TEMPO_ACAO'] = tp
        
        if (lst_task[k]['seq_task'] == total_tasks and andamento == 1):
            
            lst_task[k]['DT_ATENDIMENTO'] = dt_atendimento if tem_dt_recebimento == 1 else None
            lst_task[k]['DT_FIM'] = None

        else:
            lst_task[k]['DT_ATENDIMENTO'] = dt_atendimento
            lst_task[k]['DT_FIM'] = ult_dt_conclusão
                
        lst_task[k]['GAP'] = gap    
        lst_task[k]['ENCAMINHADO'] = gap
        lst_task[k]['ENCAMINHADO'] = rd.sample(deps, k = 1)[0]
        
        ult_dt_conclusão = ult_dt_inicio + dt.timedelta(days = -gap)
        
    return lst_task

In [ ]:
lstent_all_protocolo = []

# para cada entidade gera uma coleção de serviços e de usuários
for idx, i in enumerate(lst):
    
# protocolos
    lstent_protocolo = []
# estipula a quantidade de protocolos será simulado para esta entidade
    qtd = rd.randint(pr_trans['protocolo']['min'], pr_trans['protocolo']['max'])

    # define a coleção de protocolos para esta entidade
    for k in range(qtd):
        # define o código do protocolo
        codigo_protocolo = str(idx + 1) + '.pr-' + str(k + 1)
        
        # determina por sorteio o status do protocolo
        status = rd.choices(pr_trans['protocolo']['prop_desc_status'], 
                            pr_trans['protocolo']['prop_status'], k = 1)[0]

#         # determina por sorteio se o protocolo é cancelado
#         x = pr_trans['protocolo']['prop_cancelados'][0]; y = pr_trans['protocolo']['prop_cancelados'][1]
#         cancelado = rd.choices([0, 1], [x, y], k = 1)[0]
        
#         # determina por sorteio se o protocolo é encerrado
#         x = pr_trans['protocolo']['prop_encerrados'][0]; y = pr_trans['protocolo']['prop_encerrados'][1]
#         encerrado = rd.choices([0, 1], [x, y], k =1 )[0]
        
        # determina o status do protocolo ("Em Andamento", "Cancelado", "Concluído")
        sit = Ret_Situacao(encerrado, cancelado)
        
        # determina o serviço, a categoria de serviços e o grupo responsável do protocolo
        serv_protocolo = rd.sample(i['servicos'], k = 1)[0]
        
        # determina o usuário e grupo de usuário do protocolo
        user_protocolo = rd.sample(i['usuarios'], k = 1)[0]        
        
        # determina por sorteio a quantidade de tasks do protocolo
        qtd_tasks = rd.randint(sit[2], sit[3])
        
        # monta a estrutura da task
        dic = {'cod_protocolo': codigo_protocolo,
               'cancelado': cancelado,
               'encerrado': encerrado,
               'situacao': sit,
               'qtd_tasks': qtd_tasks,
               'servico': serv_protocolo,
               'usuario': user_protocolo
              }

        # tasks
        dic['tasks'] = Gera_Task(
                                 dic,
                                 i['departamentos'],
                                 i['acoes'], 
                                 (pr['tasks_por_protocolo']['tempos']['min'], 
                                  pr['tasks_por_protocolo']['tempos']['max']),
                                 (pr['tasks_por_protocolo']['gap_entre_tasks']['min'], 
                                  pr['tasks_por_protocolo']['gap_entre_tasks']['max']),
                                 (pr['tasks_por_protocolo']['gap_atendimento']['min'], 
                                  pr['tasks_por_protocolo']['gap_atendimento']['max'],
                                  pr['tasks_por_protocolo']['gap_atendimento']['prop_anda_atend'])
                        )
        
        lstent_all_protocolo.append(dic)
        lstent_protocolo.append(dic)
        
    lst[idx]['protocolos'] = lstent_protocolo
# lstent_all_protocolo

In [ ]:
lst[0]

# <font color='black'>gera Arquivo Externo</font>

In [ ]:
lst_tmp = []
for i in lst:
    for k in i['protocolos']:
        for m in k['tasks']:
            lst_tmp.append(
                [i['entidade'],                            # Entidade
                 'GD_' + i['entidade'].replace(' ', '_'),  # Entidade - código
                 k['servico']['servico'],                  # Serviço
                 k['cod_protocolo'],                       # Protocolo
                 k['usuario']['usuario'],                  # Usuário
                 k['usuario']['grupo_usuario'],            # Grupo
                 m['DT_FIM'],                              # Data e Hora da conclusão
                 m['DT_ATENDIMENTO'],                      # Data e Hora de atendimento
                 m['DT_INICIO'],                           # Data e Hora de criação
                 m['acao'][0],                             # Ação
                 m['ENCAMINHADO'],                         # Encaminhado para
                 None,                                     # Comentário
                 m['acao'][1]                              # Apoio
                ]                                
            )
lst_tmp

In [ ]:
len(lst_tmp)

# <font color='black'>TESTE</font>